# Build Advisor PoC

AI Applications

- Audience is technical
- LLM + good prompt engineering

Action Item:
- Work with Mike to get a few shot examples (3+) for each of
  - Definition agent
  - Specification agent

W1    
  - Deploy on HF spaces
      - Landing Page Intro: "I am... I do... how to use me..."

W2
  - Create PDF output
  - open PDF view
  - download PDF
  - feedback box
      - good/bad
      - would you like to add/change? (Y/N)
        - if Y: what would you like to add/change:
        - if N: exit

Iteration (pitfall) - work section-by-section w/ saved states
how do you change a specific part based on feedback

### Business Value: 🚦Green Light Projects

## BizDev (v1)

Definition (What?) : Product Requirement Definition -> Specification (How?) : Engineering/Function -> Plan (When? Who?): Resource Allocator

⬇️📄 Project Output

___

imports

In [2]:
import instructor
from openai import OpenAI
import os
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

# Patch the OpenAI client with Instructor
client = instructor.from_openai(OpenAI(api_key=os.environ['OPENAI_API_KEY']))

User Prompt

In [3]:
user_proposal = "I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to"

LLM function

In [4]:
# Function
def llm_call(user_prompt, system_prompt=None, response_model=None):
    messages = [
        {"role": "user", "content": user_prompt}
    ] if system_prompt is None else [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=response_model,
        messages=messages,
    )

# Usage:
# response = llm_call("user prompt").choices[0].message.content if response_model is None else llm_call("user prompt", response_model=response_model).choices[0].message.content

In [5]:
llm_call("testing... 1,2,3").choices[0].message.content

'Testing complete! How can I assist you today?'

### PRD Agent
Definition (What?) : Product Requirement Definition

In [6]:
def generate_prd_from_file(user_proposal, prd_template_file='../prompts/PRD_TEMPLATE.txt'):
    # sourcery skip: inline-immediately-returned-variable
    with open(prd_template_file, 'r') as file:
        prd_template = file.read()

    prd_document = prd_template.format(user_proposal=user_proposal)
    return prd_document

# Example usage
user_proposal = "I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to"
prd_template_file = '../prompts/PRD_TEMPLATE.txt'  # Path to your PRD template file
# print(generate_prd_from_file(user_proposal, prd_template_file))


In [23]:
prd_sys1 = generate_prd_from_file(user_proposal, prd_template_file)
prd_response_raw = llm_call(user_prompt=user_proposal, system_prompt=prd_sys1).choices[0].message.content
print(prd_response_raw)

To create a portfolio chatbot as proposed, let's outline a Product Requirements Document (PRD) tailored to this project.

### Portfolio Chatbot PRD

#### Introduction
- **Objective/Goal**: Develop an interactive, user-friendly chatbot designed to answer questions regarding an individual's work experience, skills, education, and qualifications relevant to a job role they're applying for. The chatbot aims to provide potential employers or recruiters with insightful, personalized responses, simulating a conversational interview with the candidate.

#### Features

- **Feature 1: Personalized Responses**
  - **Description**: The chatbot will generate personalized responses based on the user's query about the candidate’s experience, skills, and education. It involves processing natural language inputs to provide specific details from the candidate's portfolio.
  - **Goal**: To deliver tailored information that represents the candidate effectively, improving engagement with recruiters.
  - **

In [8]:
from pydantic import BaseModel
from typing import List

class Feature(BaseModel):
    description: str
    goal: str
    use_case: str
    additional_details: str

class PRD(BaseModel):
    objective_goal: str
    features: List[Feature]
    ux_flow_design_notes: str
    system_environment_requirements: str
    assumptions: List[str]
    constraints: List[str]
    dependencies: List[str]
    prompt_engineering_practices: str
    task_composability: str
    review_approval_process: str

# Example usage with the provided PRD content
user_proposal = "I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to"

prd_content = PRD(
    objective_goal=f"Develop a chatbot that interacts with potential employers or recruiters by providing detailed responses about your professional experience, qualifications, and any relevant projects or skills related to the role you're applying for. User Proposal: {user_proposal}",
    features=[
        Feature(
            description="The chatbot will leverage NLP to understand and respond to queries about your professional background, skills, qualifications, and work experiences.",
            goal="To simulate a real-time conversation with recruiters, offering them a personalized experience.",
            use_case="A recruiter inquiring about your experience with Python will receive a detailed answer highlighting your years of experience, projects completed, and the proficiency level.",
            additional_details="The chatbot will not require personal user data from the recruiter, focusing solely on professional information."
        ),
        Feature(
            description="Tailor responses based on specific roles or requirements mentioned by the recruiter.",
            goal="To emphasize aspects of your portfolio that are most relevant to the position in question.",
            use_case="If a recruiter mentions looking for a frontend developer, the chatbot immediately highlights your expertise in HTML, CSS, JavaScript, and relevant projects.",
            additional_details="The system will include a mapping mechanism to correlate job roles with specific skills and experiences in your portfolio."
        )
    ],
    ux_flow_design_notes="The chatbot will be integrated on professional networking sites or personal portfolio pages with an inviting user interface prompting the recruiter to start the dialogue. Users initiate interaction by typing in a question or selecting pre-set queries about your qualifications. The chatbot will then guide the conversation based on the query, asking follow-up questions if necessary. Ensure the chat interface is accessible, intuitive, and does not require extensive technical knowledge to interact with.",
    system_environment_requirements="Supported Environments: Browsers: Chrome, Firefox, Safari. Operating Systems: Windows 10, macOS. Mobile responsiveness for access on smartphones and tablets.",
    assumptions=[
        "Recruiters have a basic understanding of how to interact with chatbots."
    ],
    constraints=[
        "The chatbot will rely on accurately updated information to remain relevant."
    ],
    dependencies=[
        "Integration with professional networking platforms for live data updates and LinkedIn APIs for fetching the latest portfolio updates."
    ],
    prompt_engineering_practices="Use structured data for the backend portfolio information to streamline clear, specific responses. Implement version control for the chatbot's knowledge base to keep it updated with the newest information. Regularly test the chatbot interaction flows and update based on feedback to improve conversational accuracy and relevance.",
    task_composability="The chatbot will handle multiple inquiries in a single session, prioritizing by the sequence of the conversation. Utilize AI conversational agents with integration of filtering tools to select the most relevant portfolio pieces for each question. Implement conditional logic to guide the flow of conversation based on the context and complexity of the inquiries.",
    review_approval_process="Initial Review: Your own review of the chatbot's responses to ensure accuracy. Business Side Stakeholders Review: Seek feedback from mentors or colleagues in your industry. Engineering Handoff: Detailed documentation for developers on the team, ensuring they understand the requirements and constraints. Final Approval: Test the chatbot with a group of recruiter contacts for real-world feedback and final adjustments."
)

print(prd_content.model_dump_json(indent=4))

{
    "objective_goal": "Develop a chatbot that interacts with potential employers or recruiters by providing detailed responses about your professional experience, qualifications, and any relevant projects or skills related to the role you're applying for. User Proposal: I want to create a portfolio chatbot that answers questions about my experience and qualifications for a role that I might be applying to",
    "features": [
        {
            "description": "The chatbot will leverage NLP to understand and respond to queries about your professional background, skills, qualifications, and work experiences.",
            "goal": "To simulate a real-time conversation with recruiters, offering them a personalized experience.",
            "use_case": "A recruiter inquiring about your experience with Python will receive a detailed answer highlighting your years of experience, projects completed, and the proficiency level.",
            "additional_details": "The chatbot will not requir

In [28]:
prd_sys1 = generate_prd_from_file(user_proposal, prd_template_file)
prd_class_response = llm_call(user_prompt=user_proposal, system_prompt=prd_sys1, response_model=PRD)
print(prd_class_response)

objective_goal="Create a portfolio chatbot to effectively communicate the user's experience and qualifications relevant to the roles they are applying for." features=[Feature(description='Interactive Chat Interface', goal="To enable potential employers to engage directly with the chatbot in a conversational manner, asking questions about the user's experience, skills, and qualifications.", use_case="A potential employer visiting the user's portfolio website can initiate a conversation with the chatbot to learn more about the user's professional background and achievements.", additional_details="This feature should support basic natural language processing to understand and respond to a wide range of queries. Out-of-scope items include the chatbot's ability to engage in discussions outside of the predefined portfolio context."), Feature(description='Dynamic Content Generation', goal="Automatically update the chatbot responses based on the latest achievements and qualifications added to 

In [29]:
print(prd_class_response.model_dump_json(indent=4))

{
    "objective_goal": "Create a portfolio chatbot to effectively communicate the user's experience and qualifications relevant to the roles they are applying for.",
    "features": [
        {
            "description": "Interactive Chat Interface",
            "goal": "To enable potential employers to engage directly with the chatbot in a conversational manner, asking questions about the user's experience, skills, and qualifications.",
            "use_case": "A potential employer visiting the user's portfolio website can initiate a conversation with the chatbot to learn more about the user's professional background and achievements.",
            "additional_details": "This feature should support basic natural language processing to understand and respond to a wide range of queries. Out-of-scope items include the chatbot's ability to engage in discussions outside of the predefined portfolio context."
        },
        {
            "description": "Dynamic Content Generation",
    

### Making a change to section of the PRD (possibly a v2 feature)

In [30]:
import json
prd_json = json.loads(prd_class_response.model_dump_json(indent=4))

In [21]:
section_to_change_key = "ux_flow_design_notes"
section_to_change_val = prd_json["ux_flow_design_notes"]

# edit info

change_template = f"Change this section, with name: {section_to_change_key}\nwith content: {section_to_change_val}\n\nto reflect user feedback.\n\n"
def change_function(change_instruction):
    # sourcery skip: inline-immediately-returned-variable
    change_instruction_full = change_template + change_instruction
    change_response = llm_call(change_instruction_full)
    return change_response.choices[0].message.content

changed_response = change_function("The UI is going to use Chainlit. See link: https://docs.chainlit.io/get-started/overview")
print(changed_response)

### **ux_flow_design_notes**

Based on user feedback, it is clear that prioritizing a seamless and intuitive UX flow remains paramount. To enhance users' ability to effortlessly input information and engage with the chatbot, the following adjustments are essential:

- **Integration with Chainlit:** The UI will now leverage Chainlit, as it offers robust, user-friendly interfaces that can significantly improve the chatbot interaction experience. For more information, refer to the Chainlit documentation: [Chainlit Overview](https://docs.chainlit.io/get-started/overview).

- **Minimalistic Design Philosophy:** Keeping the design minimalistic has been well-received. However, it's crucial to ensure that this simplicity doesn't detract from the user's ability to navigate effectively. The design will maintain its clean aesthetic while enhancing visibility for essential tasks.

- **Enhanced Instructions and Feedback:** Users indicated a desire for clearer instructions and immediate feedback on 

### Designer Agent
Specification (How?) : Engineering/Function

In [38]:
designer_prompt = f"""
<System Message>
You are a the Engineer/Designer Agent a responsible for creating a functional product.
For a given proposal and PRD, write the simplest python script to quickly test your PoC. Only output python code and markdown.
</System Message>
\n<>####<>\n
<Context>
Proposed PRD:
{prd_response_raw}
\n<>####<>\n
Proposed PRD in its pydantic class form:
{prd_class_response.model_dump_json(indent=4)}
</Context>
\n<>####<>\n
<Original User Proposal>
Original User Proposal:
{user_proposal}
</Original User Proposal>
\n<>####<>\n
<Instruction>
Guide the following instructions with the above system message and above context specifications as well as the original user proposal.
Write a simple AI application development plan for the user with as few steps as possible. Do not be verbose and explain the plan in an elementary school level.
The instructions to develop this Proof of Concept Implementation should only take one engineer about a day or two to complete.
Make sure what you are suggesting exist, so for each suggestion think step by step why you are suggesting it and how to find resources for that component.
Go to the below link, read it and use it as a reference guide for starting out:
https://raw.githubusercontent.com/AI-Maker-Space/Beyond-ChatGPT/main/README.md
</Instruction>
\n<>####<>\n
"""

In [41]:
# Designer Agent Function
def designer_agent(designer_prompt):
    return llm_call(designer_prompt)

designer_output = designer_agent(designer_prompt)
print(designer_output.choices[0].message.content)

```python
# AI Portfolio Chatbot: Simple Proof of Concept Implementation Plan

# Step 1: Environment Setup
# - Ensure Python 3.8+ is installed on your machine.
# - Install necessary libraries: Flask for web app, requests for API calls, and transformers for NLP.

# Step 2: Create a Flask Web Application
# - Flask is used to set up an interactive interface for the chatbot.
# - Create a basic HTML template to communicate with the chatbot.

from flask import Flask, request, jsonify, render_template

app = Flask(__name__)

# Step 3: Integrate Simple NLP (Natural Language Processing)
# - Use Hugging Face's transformers library to utilize a pre-trained NLP model for understanding and generating text.
# - This will help parse user queries and formulate responses that resemble conversation.

from transformers import pipeline

nlp = pipeline("text-generation", model="gpt2")  # A simple model for demo purposes

# Step 4: Define the Chatbot's Response Logic
# - Implement a function to handle chat 